In [18]:
import cv2
import math
import cv2 as cv
import numpy as np
import sys
from random import randint

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ Wheel ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
class OptFlow:
    def __init__(self, resize_width=800, resize_height=400, height_start=0.2, height_end=0.5):
        self.width = resize_width
        self.height = resize_height

        self.height_start = int(self.height * height_start)
        self.height_end = int(self.height * height_end)

    def get_direction(self, frame1, frame2):
        frame1 = cv2.resize(frame1, (self.width, self.height))
        frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2 = cv2.resize(frame2, (self.width, self.height))
        frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(frame1[self.height_start:self.height_end],
                                            frame2[self.height_start:self.height_end], None, 0.5, 3, 15, 1, 5, 1.2, 0)
        flow_avg = np.median(flow, axis=(0, 1))

        move_x = -1 * flow_avg[0]
        move_y = -1 * flow_avg[1]

        return move_x, move_y
    
flow = OptFlow()





#cap = cv2.VideoCapture("data/testvideo.mp4")      # 1920 1080
cap = cv2.VideoCapture("data/kart.avi")            # 1024 768
#cap = cv2.VideoCapture("C:\\Users\\MiRcomputer\\Documents\\카트라이더\\비디오\\kart 2019-08-19 10-41-28-949.avi")
Multitracker = cv2.MultiTracker_create()     # MultiTracker 객체 생성
#cv2.namedWindow("LineTrackingWindow")





# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ ROI ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

# 첫번째 프레임 읽기
ok, src = cap.read()
if not ok:
    print('Failed to read video')
    exit()    # 커널 종료 
    
bboxes = []
n = 0
while n<1:       # ROI 1번 설정    ->   3번설정: n < 3 
    bbox = cv2.selectROI('Source', src)   # BoundingBox 설정 
    bboxes.append(bbox)
    n += 1

for bbox in bboxes:
    Multitracker.add(cv2.TrackerMIL_create(), src, bbox)  # Multitracker.add(알고리즘, 이미지, bbox)
    
'''
OPENCV_OBJECT_TRACKERS = {
"csrt": cv2.TrackerCSRT_create,      # More accurate than KCF but slower. 낮은 프레임(fps25)에서 사용 (Best)
"kcf": cv2.TrackerKCF_create,        # Fast and accurate. but 가림현상을 잘 처리하지 못함.
"boosting": cv2.TrackerBoosting_create,  # 오래된 추적기로 성능이 떨어짐.
"mil": cv2.TrackerMIL_create,            # boosting에 비해 정확도는 좋으나, 실패보고가 정상적으로 안됨.
"tld": cv2.TrackerTLD_create,       # 가림현상에 잘 작동하나, Android버전에서 에러빈번하게 발생.
"medianflow": cv2.TrackerMedianFlow_create,   # 실패보고가 잘 작동하나, 빠른 변화에 취약함 
"mosse": cv2.TrackerMOSSE_create,     # Extremely fast but not as accurate as either KCF or CSRT
"goturn": cv2.TrackerGOTURN_create    # 딥러닝 기반. 추가 모델 필요.
}
'''

colors = []
colors.append( (randint(0, 255),randint(0, 255), randint(0, 255)) )      # 랜덤컬러 3번설정 
#colors.append( (randint(0, 255),randint(0, 255), randint(0, 255)) )
#colors.append( (randint(0, 255),randint(0, 255), randint(0, 255)) )

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ Wheel ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
ok, img1 = cap.read()
ok, img2 = cap.read()


while (True):
    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ Wheel ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    x, y = flow.get_direction(img1, img2)
    STEER = x*50

    # print('STEER: {}'.format(STEER))        # 핸들 꺾는 각도 
    if int(STEER) < 0 :
        print('LEFT')

    if int(STEER) > 0 :
        print('RIGHT')

    #cv2.imshow('result', img1)
    wheel = cv2.imread('data/wheel_rion.png')
    wheel = cv2.resize(wheel, dsize=(335, 335), interpolation=cv2.INTER_AREA)

    M = cv2.getRotationMatrix2D((335 / 2, 335 / 2), -int(STEER), 1)

    wheel = cv2.warpAffine(wheel, M, (335, 335))

    cv2.imshow('wheel', wheel)

    img1 = img2
    _, img2 = cap.read()
    img2 = cv2.resize(img2, dsize=(640, 360), interpolation=cv2.INTER_AREA)
    
    
    
    
    
    ret, src = cap.read()    # frame 읽어오기
    
    if not ret:
        print ('no image to read')
        break
        
    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ ROI ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ    
    ## Start timer
    timer = cv2.getTickCount()

    ok, boxes = Multitracker.update(src)  # frame에서 설정한 boundingBox와 비슷한 물체의 위치를 
                                            # 찾아(Update) 반환한다.
    
    ## Calculate Frames per second (FPS) 
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
    
    i = 0
    for newbox in boxes:
        # newbox의 왼쪽, 위쪽, 너비, 높이
        left, top, w, h = [int(v) for v in newbox]
        
        # boxes의 색깔,두께 설정
        cv2.rectangle(src, pt1=(left, top), pt2=(left + w, top + h), color=colors[i], thickness=3)   # 이미지에 바운딩박스 그린다.
        i += 1

    # Display FPS on frame
    # Green : (50,170,50)
    cv2.putText(src, "FPS : " + str(int(fps)), (400,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 4)
    
    #cv2.imshow('LineTrackingWindow', src)
    
    # ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ Line ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
    src = cv2.resize(src, (1000, 650))              # 원본 Window
    src_line_window = cv2.resize(src, (300, 200))     # Line Window

    low_threshold = 800                     # 기본: 50 200 150
    high_threshold = 920
    dst = cv.Canny(src_line_window, low_threshold, high_threshold, None, 
                   apertureSize=3)   # 엣지 검출 알고리즘     
    
    '''
    canny = cv2.Canny(src, 100, 255)
    sobel = cv2.Sobel(gray, cv2.CV_8U, 1, 0, 3)
    laplacian = cv2.Laplacian(gray, cv2.CV_8U, ksize=3)
    '''
    
    cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
    cdstP = np.copy(cdst)

    rho = 1
    theta = np.pi / 180
    threshold = 58
    
    #min_line_length = 65
    #max_line_gap = 5
    
    lines = cv.HoughLines(dst, rho, theta, threshold)   # 허프변환을 통한 라인검출 알고리즘    # Standard Hough Line Transform
    #lines = cv.HoughLines(dst, 1, np.pi / 180, 150, None, min_line_length, max_line_gap)   # Probabilistic Line Transform
    
    if lines is not None:
        for i in range(0, len(lines)):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))    # FirstPoint of line Segment
            pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))    # SecondPoint of line Segment
            cv2.line(cdst, pt1, pt2, (0, 0, 255), 3, cv2.LINE_AA)   # cv2.line(img, pt1, pt2, color, thickness, LINE_AA) 
    
    '''
    linesP = cv.HoughLinesP(dst, rho, theta, threshold, None, min_line_length, max_line_gap)

    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            pt1 = (l[0], l[1])
            pt2 = (l[2], l[3])
            cv.line(cdstP, pt1, pt2, (0, 0, 255), 3, cv.LINE_AA)
    '''



    cv.imshow("Source", src)
    cv.imshow("Detected Lines (in red) - Standard Hough Line Transform", cdst)
    #cv.imshow("Detected Lines (in red) - Probabilistic Line Transform", cdstP)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
RIGHT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
RIGHT
RIGHT
RIGHT
RIGHT
LEFT
LEFT
RIGHT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
RIGHT
RIGHT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
LEFT
LEFT
RIGHT
RIGHT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
RIGHT
R

## reference
```
<http://makeshare.org/bbs/board.php?bo_table=raspberrypi&wr_id=10>

처음에 엣지 검출을 통해서 상당한 수의 자잘한 세그먼트들을 필터링한다면 
이후에 이루어질 허프 변환 처리 속도가 빨라질 수 있다. 

그리고, lines = cv2.HoughLines(edges,1,np.pi/180,170)가 허프변환을 통해 라인을 검출하는 알고리즘이며, 
170이라는 숫자가 올라갈 수록 많은 교점이 있는 직선을 검출한다. 

```

In [1]:
import cv2
cv2.HoughLines?